In [1]:
import os
import pandas as pd

In [127]:
import chardet
# Encoding der Datei automatisch erkennen
with open('./Outputs_lists/Output_list3rev_1.txt', 'rb') as f:
    result = chardet.detect(f.read())
    print(result['encoding'])

utf-8


In [4]:
# util.js DATEI ERSTELLEN

import os
import pandas as pd
import chardet

# Funktion zur automatischen Encoding-Erkennung
def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    return result['encoding']

# Directory containing the text files
directory = "./Outputs_lists"

# Keys to label the columns
keys = ['Trial_no','Trial_cat','Cat_comb_type','Word1','Word2','Pair_no1','Pair_no2','Gender_comb','Cat_comb','']

# Pfad zur util.js Datei
util_js_path = 'util.js'

# Lies die existierende util.js Datei
with open(util_js_path, 'r') as util_file:
    util_js_content = util_file.readlines()

# Finde den Index der Zeile mit dem Kommentar // INSERT_HERE
insert_index = None
for i, line in enumerate(util_js_content):
    if "// INSERT_HERE" in line:
        insert_index = i
        break

if insert_index is None:
    print("Kommentar // INSERT_HERE nicht gefunden.")
else:
    # Öffne die util.js Datei zum Schreiben
    with open(util_js_path, 'w') as util_file:
        # Schreibe alle Zeilen bis zur // INSERT_HERE Stelle
        for line in util_js_content[:insert_index + 1]:
            util_file.write(line)

        # Erstelle den JavaScript-Code, den du einfügen möchtest
        util_file.write("var lists = {\n")

        # Loop durch alle Dateien im Verzeichnis
        for i, filename in enumerate(os.listdir(directory)):
            if filename.endswith(".txt"):  # Nur .txt Dateien verarbeiten
                filepath = os.path.join(directory, filename)

                try:
                    # Lese die Textdatei in ein DataFrame
                    enc = detect_encoding(filepath)
                    df = pd.read_csv(filepath, sep='\t', header=None, encoding=enc)#'ISO-8859-1''windows-1252')

                    df = df.fillna("")

                    # Entferne die letzte Spalte aus der Output-Datei
                    if df.shape[1]==10:
                        df = df.iloc[:, :-1]
                        print('removed last column from '+filename)

                    df[3] = df[3].str.upper()
                    df[4] = df[4].str.upper()

                    # Erstelle die concatenated column im gewünschten Format
                    df['Concatenated_Row'] = df.apply(
                        lambda row: '{ ' + ', '.join([
                            f'{keys[i]}: "{int(value) if isinstance(value, float) and value.is_integer() else value}"'
                            for i, value in enumerate(row)
                        ]) + ', Type: jsPsychSurveyText }', axis=1
                    )

                    # Hole den Namen für die Liste aus dem Dateinamen, entferne 'Output_'
                    list_name = filename.replace('Output_', '').replace('.txt', '')

                    # Schreibe die Liste in die util.js Datei im JavaScript-Format
                    util_file.write(f"  {list_name}: [\n")
                    for row in df['Concatenated_Row']:
                        util_file.write(f"    {row},\n")
                    util_file.write("  ],\n")
                    
                except Exception as e:
                    print(f"Fehler beim Verarbeiten von {filename}: {e}")

        # Ende des JavaScript-Objekts
        util_file.write("};\n")

        # Schreibe alle Zeilen nach // INSERT_HERE
        for line in util_js_content[insert_index + 1:]:
            util_file.write(line)




removed last column from Output_list1_1.txt
removed last column from Output_list1_10.txt
removed last column from Output_list1_2.txt
removed last column from Output_list1_3.txt
removed last column from Output_list1_4.txt
removed last column from Output_list1_5.txt
removed last column from Output_list1_6.txt
removed last column from Output_list1_7.txt
removed last column from Output_list1_8.txt
removed last column from Output_list1_9.txt


In [136]:
#CONFIG DATEI ERSTELLEN

import os

# Das Verzeichnis, das durchlaufen werden soll
directory = 'Outputs_lists'

# Der Name der Konfigurationsdatei
config_filename = './config/experiment.config'

# Öffne die Datei zum Schreiben (falls sie nicht existiert, wird sie erstellt)
with open(config_filename, 'w') as config_file:
    header = """
    {
        "name": "Traumgestalten",
        "notes": "Pilotstudie zu sprachlicher Kreativität C01",
        "completion_URL": "https://app.prolific.com/submissions/complete?cc=C3A361EJ",
        "conditions": [
        """
    
    config_file.write(header)

    # Durchlaufe alle Dateien im Verzeichnis
    for filename in os.listdir(directory):
        # Überprüfe, ob es sich um eine Datei handelt
        if os.path.isfile(os.path.join(directory, filename)):
            # Schreibe den Dateinamen in die Konfigurationsdatei
            listname = filename.replace('Output_','').replace('.txt','')

            def cond(listname):
                output = f"""
                {{
                    "name": "{listname}",
                    "experiment_URL": "https://lisahud.github.io/C01/experiment.html?list={listname}",
                    "participants": 1
                }},
                """
                return output

            # Datei schreiben
            config_file.write(cond(listname))

    config_file.write(']\n}')


print(f"Dateien aus '{directory}' wurden in '{config_filename}' geschrieben.")


Dateien aus 'Outputs_lists' wurden in './config/experiment.config' geschrieben.
